# We can also load and explore the model data from there as well

In [ ]:
import xarray as xr
import zarr
import matplotlib.pyplot as plt
import numpy as np
from cartopy import crs as ccrs
from fv3viz import pcolormesh_cube

We can look at the column-integrated water vapor and it's evolution over the model run:

In [ ]:
ds = xr.open_zarr(store=zarr.DirectoryStore(path="output_prerun.zarr"), consolidated=False)
nx=48
nz = 32
rotation = -120.

fig, ax = plt.subplots(2, 3, figsize=(12, 8))
ax = ax.flatten()
level = -1
varname = "qvapor"
for i in range(6):
    column_q = (
        np.sum(ds[varname].isel(time=-1, tile=i).values, axis=-1)
    )
    im = ax[i].pcolormesh(column_q)
    ax[i].set_title(f"Tile {i}")
    plt.colorbar(im, ax=ax[i])
fig.suptitle("end step column integrated water vapor")
plt.tight_layout()
plt.show()

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection=ccrs.Robinson())
f1 = pcolormesh_cube(
    ds["lat"].values * 180./np.pi,
    ds["lon"].values * 180./np.pi,
    np.sum(ds["qvapor"].isel(time=-1).values, axis=-1),
    cmap="viridis",
)
plt.colorbar(f1, label="water vapor")
ax.set_title(f"Column-Integrated Water Vapor ({nx} x {nx} points per tile)")

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection=ccrs.Robinson())
f1 = pcolormesh_cube(
    ds["lat"].values * 180./np.pi,
    ds["lon"].values * 180./np.pi + rotation,
    np.sum(ds["qvapor"].isel(time=-1).values, axis=-1) - np.sum(ds["qvapor"].isel(time=1).values, axis=-1),
    cmap="viridis",
)
plt.colorbar(f1, label="water vapor change")
ax.set_title(f"Column-Integrated Water Vapor Evolution ({24} x {24} points per tile)")


fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection=ccrs.Robinson())
f1 = pcolormesh_cube(
    ds["lat"].values * 180./np.pi,
    ds["lon"].values * 180./np.pi + rotation,
    ds["pt"].isel(time=-1, z=-2).values,
    cmap="viridis",
)
plt.colorbar(f1, label="Temperature (K)")
ax.set_title(f"Temperature ({nx} x {nx} points per tile)")

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection=ccrs.Robinson())
f1 = pcolormesh_cube(
    ds["lat"].values * 180./np.pi,
    ds["lon"].values * 180./np.pi,
    np.sum(ds["qcld"].isel(time=-1).values, axis=-1)/nz,
    cmap="viridis",
)
plt.colorbar(f1, label="cloud mixing ratio")
ax.set_title(f"Column-Integrated Cloud Mixing Ratio ({nx} x {nx} points per tile)")

We can also look at the temperature at each atmospheric level:

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection=ccrs.Robinson())
f1 = pcolormesh_cube(
    ds["lat"].values * 180./np.pi,
    ds["lon"].values * 180./np.pi + rotation,
    (ds["pt"].isel(time=-1, z=-2).values - ds["pt"].isel(time=1, z=-2).values),
    cmap="viridis",
)
plt.colorbar(f1, label="change in temperature (K)")
ax.set_title(f"Change in 850 hPa Temperature ({24} x {24} points per tile)")

Or liquid water:

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection=ccrs.Robinson())
f1 = pcolormesh_cube(
    ds["lat"].values * 180./np.pi,
    ds["lon"].values * 180./np.pi + rotation,
    np.sum(ds["qliquid"].isel(time=-1).values, axis=-1) - np.sum(ds["qliquid"].isel(time=1).values, axis=-1),
    cmap="viridis",
)
plt.colorbar(f1, label="liquid water evolution")
ax.set_title(f"Column-Integrated Liquid Water Evolution ({24} x {24} x {6} resolution)")